# Content Audit [Keep / Refresh / Redirect Mapping]

[INSERT VIDEO WALKTHROUGH]

In [1]:
#@title Mount Drive to the Notebook { display-mode: "form" }
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
#@title Import API Module from /My Drive/Colab Notebooks/ { display-mode: "form" }
import sys
sys.path.insert(0, "drive/My Drive/Colab Notebooks/")
# sys.path.insert(0, "drive/My Drive/GSC_custom_api_library")
from api import *
colab_path = "/content/drive/My Drive/Colab Notebooks/" 
# colab_path = "/content/drive/My Drive/GSC_custom_api_library"

Go to the following link in your browser: https://accounts.google.com/o/oauth2/v2/auth?client_id=733455143894-uob3vcm050frdb8c0fd4qjetg2lrv9tr.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fwebmasters.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fanalytics.readonly&access_type=offline&response_type=code
Enter verification code: 4/1AY0e-g5qErFoFo4tu173lA2yFAoU_OAWUlgIU-gFbM6nrqqxMbvtm72csac
Credentials saved for later.


In [3]:
#@title Import External Dependencies { display-mode: "form" }
import pandas
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup, NavigableString
import requests
import random
import lxml
import lxml.html
import lxml.etree
from lxml.etree import ParseError
from lxml.etree import ParserError
from urllib.parse import urlparse
import os
from google.colab import data_table
from IPython.display import HTML

<br>

# Input Variables [for Extraction + Variables] 

<br>

In [4]:
#@markdown #  Set `xpath_selector` to scrape MAIN CONTENT ONLY { run: "auto", display-mode: "form" }
#@markdown *Almost all sites have multiple templates - combine xpaths for each template using "|").    
#@markdown E.g., `//div[@class='example-1']|//div[@class='example-2']|//div[@class='example-3']`

xpath_selector = "//div[@class='entry-content']|//div[@class='body']" #@param {type: "string"}

#@markdown ---

#@markdown # Search Console Click KPIs - range for quality / value considerations 
#@markdown Low range [not much traffic]  
low_clicks_1 = 1  #@param {type: "integer"}
low_clicks_2 = 99  #@param {type: "integer"}
#@markdown Strong range [strong traffic]  
strong_clicks_1 = 100  #@param {type: "integer"}
strong_clicks_2 = 1000000  #@param {type: "integer"}

#@markdown ---

#@markdown # Word Count of Main Content - range for quality / value considerations 
#@markdown Thin range [not much traffic]  
thin_words_1 = 0  #@param {type: "integer"}
thin_words_2 = 100  #@param {type: "integer"}
#@markdown Okay range [some traffic]  
okay_words_1 = 101  #@param {type: "integer"}
okay_words_2 = 999  #@param {type: "integer"}
#@markdown Strong range [strong traffic]  
strong_words_1 = 1000  #@param {type: "integer"}
strong_words_2 = 100000  #@param {type: "integer"}

<br>

# Input Variables [for Data Pull] 

<br>

In [5]:

#@markdown ### Set `domain_lookup` to match the homepage URL
domain_lookup = 'https://mint.intuit.com/'  #@param {type: "string"}
#@markdown ---
#@markdown ### Set `gsc_property` to match the GSC UI property name 
#@markdown (*For <b>"domain properties"</b> use format "sc-domain:domain.com")  
gsc_property = 'https://mint.intuit.com/'  #@param {type: "string"}
#@markdown ---
#@markdown ### Set `startdate` and `enddate` to desired analysis range 
#@markdown (*Recommend no longer than 3 months from present due to shifting query rankings)  
startdate = '2021-01-01'  #@param {type: "date"}
enddate = '2021-04-30'  #@param {type: "date"}
#@markdown ---
#@markdown ### Set page <b>INCLUSIONS</b> with format *urlID1|urlID2|...*
#@markdown (*Leave blank if no exclusions needed)
page_inclusions = "blog" #@param {type: "string"}
#@markdown ---

import os

domain_name = domain_lookup.split("www.")[-1].split("//")[-1].split(".")[0]
date = datetime.today().strftime('%Y-%m-%d')

domain_clean = urlparse("{}".format(domain_lookup)).netloc
path_step1 = "{}/{}/".format(colab_path,domain_clean)
path = "{}/{}/Content_Audit_{}".format(colab_path,domain_clean,date)
path_rawData = "{}/{}/Raw Data (Archive)".format(colab_path,domain_clean,date)
date = datetime.today().strftime('%Y-%m-%d')

try:
    os.mkdir(path_step1)
except OSError:
    print ("Creation of the directory %s failed" % path_step1)
else:
    print ("Successfully created the directory %s " % path_step1)

try:
    os.mkdir(path_rawData)
except OSError:
    print ("Creation of the directory %s failed" % path_rawData)
else:
    print ("Successfully created the directory %s " % path_rawData)

try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:    print ("Successfully created the directory %s " % path)


Creation of the directory /content/drive/My Drive/Colab Notebooks//mint.intuit.com/ failed
Creation of the directory /content/drive/My Drive/Colab Notebooks//mint.intuit.com/Raw Data (Archive) failed
Creation of the directory /content/drive/My Drive/Colab Notebooks//mint.intuit.com/Content_Audit_2021-05-31 failed


# Automated Script - No Other Manual Changes Are Needed
### Open tab to see additional details

<h2>PART 1: Get GSC Data </h2>

In [6]:
#@title Collect data from Google Search Console { display-mode: "form" }
domain_name = domain_lookup.split("www.")[-1].split("//")[-1].split(".")[0]

# Options: 'date,' 'device,' 'page,' , 'query' and "country"
dimensions=['page']

gsc_df = gscservice.get_site_data(
    gsc_property,
    startdate=startdate,
    enddate=enddate,
    dimensions=dimensions,
    output_fn="{}/{}/Raw Data (Archive)/rawData_{}_{}_{}_by_{}.csv".format(colab_path,domain_clean,domain_name, startdate.replace("-",""), enddate.replace("-",""), '_'.join(dimensions))
)

# Filter to only non-brand && specified page type (if any)
# kw_filter = ~gsc_df["query"].str.contains("{}".format(brand_exclusions), case = False, regex=True)
# gsc_df = gsc_df[kw_filter]
filter_pageType = gsc_df["page"].str.contains("{}".format(page_inclusions), case = False, regex=True)
gsc_df = gsc_df[filter_pageType]

gsc_df_clean = (
    gsc_df.groupby(["page"])
    .agg({"clicks": "sum"})
    .reset_index()
)

datasetID = "{}_{}_{}_by_{}".format(domain_name, startdate.replace("-",""), enddate.replace("-",""), '_'.join(dimensions))
gsc_df_clean.insert(loc = 0, column = "gsc_datasetID", value = datasetID)
gsc_df_clean.insert(loc = 1, column = "domain", value = domain_lookup)

Reloading Existing: /content/drive/My Drive/Colab Notebooks//mint.intuit.com/Raw Data (Archive)/rawData_mint_20210101_20210430_by_page.csv


In [7]:
#@title Calculate Clicks Range and Value { display-mode: "form" }
conds = [
    gsc_df_clean.clicks == 0,
    gsc_df_clean.clicks.between(int(low_clicks_1),int(low_clicks_2)),
    gsc_df_clean.clicks.between(int(strong_clicks_1),int(strong_clicks_2))
]

clicks_range = [
    '0',
    f'{low_clicks_1} to {low_clicks_2}', 
    f'{strong_clicks_1}+'
]
clicks_value = [
    'None',
    'Low',
    'Strong'
]

gsc_df_clean['clicks_range'] = np.select(conds,clicks_range)
gsc_df_clean['clicks_value'] = np.select(conds,clicks_value)

# del gsc_df['index']

excel_file = pandas.ExcelWriter(os.path.join(path,"final-content-audit_{}-{}.xlsx".format(domain_name,date)))

# Preview the export from GSC API
# gsc_df.to_csv(os.path.join(path,"step1_raw-page-query-data_{}-{}.csv".format(domain_name,date)),  index=False)
# gsc_df.to_excel(
#     excel_file,
#     sheet_name='GSC',
#     header=True)
# data_table.DataTable(gsc_df, include_index=False, num_rows_per_page=10)

In [8]:
#@title DEV - keep top 10 pages by clicks { display-mode: "form" }
# gsc_df_clean = gsc_df_clean.sort_values(by="clicks", ascending=False).reset_index().head(10)
# gsc_df_clean

In [ ]:
#@title Scrape Pages & Calculate Words Range and Value { display-mode: "form" }
### [BELOW AGGREGATES PAGES FOR SCRAPER]

def GET_UA():
  uastrings = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1500.72 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10) AppleWebKit/600.1.25 (KHTML, like Gecko) Version/8.0 Safari/600.1.25",
        "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",
        "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/600.1.17 (KHTML, like Gecko) Version/7.1 Safari/537.85.10",
        "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko",
        "Mozilla/5.0 (Windows NT 6.3; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.104 Safari/537.36"
  ]
  return random.choice(uastrings)

crawl_res = []

# for each page in the top N list we are going to scrape the html content
for index, row in gsc_df_clean.iterrows():
  try:
    USER_AGENT = GET_UA()
    headers = {'user-agent': USER_AGENT}
    resp = requests.get(row.to_dict()['page'], headers=headers)
    # parse the HTML with Beautiful soup
    if resp.status_code == 200:
        if xpath_selector is not None and xpath_selector != '':
            tree = lxml.html.fromstring(resp.content)
            # Get element using XPath 
            selector = f"string-length({xpath_selector}) - string-length(translate({xpath_selector},' ',''))"
            xpath_selector_count = tree.xpath(selector)
            xpath_selector_text = tree.xpath(xpath_selector)
            selected_content = b'\n'.join([lxml.etree.tostring(elem) for elem in xpath_selector_text])
            # for elem in xpath_selector_text:
            #   print(elem)
            crawl_res.append(dict(
                page=row.to_dict()['page'],
                word_count=int(xpath_selector_count),
                source_text=selected_content
            ))

  # below exceptions stop errors from breaking tool
  except ParserError as pe:
    print("ParserError: Error Message - {0}".format(pe))
    pass

  except BaseException as ge:
    print("Unidentified Error - {0}".format(ge))
    pass


# transform the list of dictionaries into a dataframe to be able to work with the exisiting dataframes
crawl_df = pandas.DataFrame([c for c in crawl_res])

conds = [
    crawl_df.word_count.between(thin_words_1,thin_words_2),
    crawl_df.word_count.between(okay_words_1,okay_words_2),
    crawl_df.word_count.between(strong_words_1,strong_words_2)
]

words_range = [
    f'{thin_words_1} to {thin_words_2}', 
    f'{okay_words_1} to {okay_words_2}',
    f'{strong_words_1}+'
]
words_value = [
    'Thin',
    'Okay',
    'Strong'
]

crawl_df['words_range'] = np.select(conds,words_range)
crawl_df['words_value'] = np.select(conds,words_value)
crawl_df.to_excel(
    excel_file,
    sheet_name='Crawl',
    header=True)
data_table.DataTable(crawl_df, include_index=False, num_rows_per_page=10)

In [ ]:
#@title Merge & Prepare Data { display-mode: "form" }
output = gsc_df_clean.merge(crawl_df, how="left")

select_cols_final = [
    "gsc_datasetID",
    "domain",
    "page",
    "clicks",
    "clicks_range",
    "clicks_value",
    "word_count",
    "words_range",
    "words_value"
]

output = output[select_cols_final]

conds = [
    (output.clicks_value == clicks_value[0]) & (output.words_value == words_value[0]),
    (output.clicks_value == clicks_value[1]) & (output.words_value == words_value[0]),
    (output.clicks_value == clicks_value[2]) & (output.words_value == words_value[0]),

    (output.clicks_value == clicks_value[0]) & (output.words_value == words_value[1]),
    (output.clicks_value == clicks_value[1]) & (output.words_value == words_value[1]),
    (output.clicks_value == clicks_value[2]) & (output.words_value == words_value[1]),

    (output.clicks_value == clicks_value[0]) & (output.words_value == words_value[2]),
    (output.clicks_value == clicks_value[1]) & (output.words_value == words_value[2]),
    (output.clicks_value == clicks_value[2]) & (output.words_value == words_value[2])
]

actions = [
    'Either redirect or refresh',
    'Either redirect or refresh',
    'Refresh',

    'Either redirect or refresh',
    'Either no change, redirect or refresh',
    'Either no change or refresh',

    'Either no change or refresh',
    'No change',
    'No change'
]

merged_value = [
    f'{clicks_value[0]}/{words_value[0]}',
    f'{clicks_value[1]}/{words_value[0]}',
    f'{clicks_value[2]}/{words_value[0]}',
    
    f'{clicks_value[0]}/{words_value[1]}',
    f'{clicks_value[1]}/{words_value[1]}',
    f'{clicks_value[2]}/{words_value[1]}',

    f'{clicks_value[0]}/{words_value[2]}',
    f'{clicks_value[1]}/{words_value[2]}',
    f'{clicks_value[2]}/{words_value[2]}',
]

output['merged_value'] = np.select(conds,merged_value)
output['calculated_actions'] = np.select(conds,actions)

output.to_excel(
    excel_file,
    sheet_name='Merge',
    header=True)

data_table.DataTable(output, include_index=False, num_rows_per_page=10)

In [ ]:
with pandas.ExcelWriter(os.path.join(path,"final-content-audit_{}-{}.xlsx".format(domain_name,date))) as writer:
    output.to_excel(writer, sheet_name='Final Dataset', index=False)
    gsc_df.to_excel(writer, sheet_name='GSC Data', index=False)
    crawl_df.to_excel(writer, sheet_name='Extracted Content', index=False)
